In [1]:
import networkx as nx
import os
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
xferpath = Path(r"C:\Users\User\Documents\cfb project\data\transferportal\raw")
recpath = Path(r"C:\Users\User\Documents\cfb project\data\recruiting\original")
xferfiles = {}
recfiles = {}

for file_path in xferpath.glob('*.graphml'):
    name = file_path.stem
    G_x = nx.read_graphml(file_path)
    xferfiles[name] = G_x

for file_path in recpath.glob('*.graphml'):
    name = file_path.stem
    G_r = nx.read_graphml(file_path)
    recfiles[name] = G_r

print(f"\nSuccessfully loaded {len(xferfiles)} files from xfer and {len(recfiles)} files from rec.")



Successfully loaded 5 files from xfer and 27 files from rec.


In [ ]:
# net degree calculation

# This will store all our results
# Format: { 'year': { 'school': ratio } }
all_nd = {}

# Loop through the dictionary of graphs you just loaded
for name, G_x in xferfiles.items():
    
    # Extract the year from the filename "transfer_portal_2024"
    try:
        year = int(name.split('_')[-1])
    except:
        year = name # Fallback if the name is not as expected
        
    print(f"--- Processing {year} ---")
    
    yearly_nd = {}
    
    # Loop through every school (node) in this year's graph
    for node in G_x.nodes():
        # Get the raw in-degree and out-degree
        in_deg = G_x.in_degree(node)
        out_deg = G_x.out_degree(node)
        
        if in_deg
        
        nd = in_deg - out_deg
        
        yearly_nd[node] = nd
        
    # Add this year's results to the main dictionary
    all_nd[year] = yearly_nd

# --- Convert to a Pandas DataFrame for easy analysis ---
# Rows will be years, Columns will be schools
netdeg_df = pd.DataFrame.from_dict(all_nd, orient='index')

# Sort by year (index) just in case
netdeg_df.sort_index(inplace=True)

print("\n--- Sample of Net Degrees (DataFrame) ---")
# Display the last 5 rows (most recent years)
print(netdeg_df.tail())

--- Processing 2021 ---
--- Processing 2022 ---
--- Processing 2023 ---
--- Processing 2024 ---
--- Processing 2025 ---

--- Sample of Net Degrees (DataFrame) ---
      Missouri  UTSA  Arkansas State  Southern Miss  Miami  Wyoming  \
2021       -10     6              11              1     -1        0   
2022         1     4              -1              0     -5       -8   
2023        -5     4               5              5     -3       -3   
2024         2     5               5              7     -6       -3   
2025         1     2               6             -2      3        1   

      Eastern Michigan  Vanderbilt  Tulsa  Missouri Western State  ...  \
2021                -1          -6      2                    -1.0  ...   
2022                 1         -10      4                    -1.0  ...   
2023                 2          -6      0                     NaN  ...   
2024                 3          -1     -2                     1.0  ...   
2025                -4           2      

In [8]:
print(ratio_df.loc[2024].sort_values(ascending=False).head(10)) # Top 10 teams in 2024

Virginia Union              inf
Morehead State              inf
Nebraska Kearney            inf
Highland C.C.               inf
Southeast Oklahoma State    inf
Missouri Southern State     inf
Itawamba C.C.               inf
Carroll College             inf
Edinboro University         inf
Lake Erie                   inf
Name: 2024, dtype: float64
